# Day 16: Ticket Translation

In [1]:
import re

In [2]:
data = open("input/input-day-16.txt", "r").read().split("\n")[:-1]

# Hardcoded indices for now
rules = data[0:20]
my_ticket = data[22]
my_ticket = list(map(int, my_ticket.split(",")))
nearby_tickets = data[25:]

## Part 1

In [3]:
def isNumberInRange(number, rule):
    if (rule[0] <= number <= rule[1]):
        return True
    return False

def isNumberValid(number, rules):
    # The number has to exist in only one of the ranges in order to be valid
    for rule in rules:
        if isNumberInRange(number, rule):
            return True

    return False

pattern_1 = re.compile(r"(\d+\-\d+) or (\d+\-\d+)")
ticket_rules = []

for rule in rules:
    # Find the numbers listed in the rules
    result = pattern_1.search(rule)
    # Create a list of each rule with the min and max values
    rule_1 = list(map(int, result.group(1).split("-")))
    rule_2 = list(map(int, result.group(2).split("-")))
    # Save all rules in a single list
    ticket_rules.extend([rule_1,rule_2])


# Find invalid numbers (and valid tickets which we need for part 2)
invalid_numbers = []
valid_tickets = []
for i, ticket in enumerate(nearby_tickets):
    ticket_numbers = list(map(int,ticket.split(",")))
    
    # Go through each number in the ticket and save it if it's invalid
    is_ticket_valid = True
    for ticket_number in ticket_numbers:
        if not isNumberValid(ticket_number, ticket_rules):
            invalid_numbers.append(ticket_number)
            is_ticket_valid = False
    if is_ticket_valid:
        valid_tickets.append(ticket_numbers)
    
# Find the secret value for our answer.
# It's the sum of all the invalid numbers found
sum(invalid_numbers)

30869

## Part 2
Use the remaining valid tickets to determine which field is which.

In [17]:
import math

pattern_2 = re.compile(r"^(.*):")
rule_names = []
for rule in rules:
    result = pattern_2.search(rule)
    rule_names.append(result.group(1))

possible_rules_for_number_at_index = {}
for i in range(len(valid_tickets[0])):
    possible_rules_for_number_at_index[i] = []
    # Go through each rule name for the i:th number in each ticket untill all numbers are valid.
    # If that's the case it means that the rule works for the i:th number
    rule_found_for_number = True
    for j, rule_name in enumerate(rule_names):        
        
        jth_rule_match_ith_number = True
        
        # Check the rule against all valid tickets
        for k, valid_ticket in enumerate(valid_tickets):
            k_ticket_ith_number = valid_ticket[i]
            
            # If the i:th number for this ticket is not in the range for this rule, move on to the next rule.
            if (not isNumberInRange(k_ticket_ith_number, ticket_rules[j*2]) and
                not isNumberInRange(k_ticket_ith_number, ticket_rules[(j*2) + 1])):
                    jth_rule_match_ith_number = False
                    break
        
        # If the rule works for the i:th number on all valid tickets. It means that the rule can be applied here.
        if jth_rule_match_ith_number:
            possible_rules_for_number_at_index[i].append(rule_name)
        

The problem is that numbers at position ```i``` could have many valid rules, but each rule can only be used once. Hence we have to find the only possible match between rules and numbers

In [16]:
possible_rules_with_number_index = []
# The first element in each list is for which number position i the rules are valid
for key in possible_rules_for_number_at_index.keys():
    possible_rules_with_number_index.append([key,*possible_rules_for_number_at_index[key]])

possible_rules_with_number_index.sort(key=lambda x: len(x))
possible_rules_with_number_index[:5]

[[16, 'type'],
 [7, 'type', 'zone'],
 [5, 'class', 'type', 'zone'],
 [2, 'arrival station', 'class', 'type', 'zone'],
 [15, 'arrival station', 'class', 'row', 'type', 'zone']]

Here we can see from the first 5 entries when sorted after the number of accepted rules, that they increase by one and that all rules in previous items also exist in the latter ones. Hence we can find the only possible solution through deductive reasoning. Where we go through them in order from smallest to largest and delete duplicates found in the lists where the number of rules isn't equal to one.

In [9]:
for i, rule in enumerate(possible_rules_with_number_index):
    for j in range(i+1, len(possible_rules_with_number_index)):
        possible_rules_with_number_index[j] = [x for x in possible_rules_with_number_index[j] if x != rule[1]]

real_rules = possible_rules_with_number_index.copy()
real_rules.sort(key=lambda x: x[0])
real_rules

[[0, 'departure date'],
 [1, 'arrival track'],
 [2, 'arrival station'],
 [3, 'arrival location'],
 [4, 'duration'],
 [5, 'class'],
 [6, 'route'],
 [7, 'zone'],
 [8, 'departure station'],
 [9, 'arrival platform'],
 [10, 'departure platform'],
 [11, 'departure track'],
 [12, 'seat'],
 [13, 'wagon'],
 [14, 'price'],
 [15, 'row'],
 [16, 'type'],
 [17, 'departure location'],
 [18, 'train'],
 [19, 'departure time']]

Now look for the rules which start with ```departure```, get those numbers from your ticket. Multiply them and you have your answer!

In [10]:
rules_starting_with_departure = [x for x in real_rules if x[1].split(" ")[0] == "departure"]
rules_starting_with_departure

[[0, 'departure date'],
 [8, 'departure station'],
 [10, 'departure platform'],
 [11, 'departure track'],
 [17, 'departure location'],
 [19, 'departure time']]

In [11]:
answer = 1
for rule in rules_starting_with_departure:
    answer *= my_ticket[rule[0]]
    
answer

4381476149273